In [11]:
import pandas as pd

# Función para cargar los archivos
def cargar_archivos(padron_file, rendicion_file):
    padron_df = pd.read_excel(padron_file)
    rendicion_df = pd.read_excel(rendicion_file, skiprows=7)
    return padron_df, rendicion_df

def contar_coincidencias(nombre_rendicion, nombre_padron):
    """
    Función para contar coincidencias entre el nombre de rendición y el nombre del padrón.
    Devuelve True si hay al menos dos coincidencias, False en caso contrario.
    """
    # Separar los nombres en palabras
    palabras_rendicion = set(nombre_rendicion.split())
    palabras_padron = set(nombre_padron.split())
    
    # Contar coincidencias
    coincidencias = palabras_rendicion.intersection(palabras_padron)
    
    # Debug: Mostrar las palabras y las coincidencias
    print(f"Palabras en rendición: {palabras_rendicion}")
    print(f"Palabras en padrón: {palabras_padron}")
    print(f"Coincidencias: {coincidencias} (Total: {len(coincidencias)})")
    
    # Retornar True si hay al menos 2 coincidencias
    return len(coincidencias) >= 2

# Función para procesar los DataFrames y realizar la comparación
def procesar_comparacion(padron_df, rendicion_df):
    # Unir "Apellido" y "Nombre" en el padrón para facilitar la comparación
    padron_df['Apellido y Nombre'] = padron_df['Apellido'] + ' ' + padron_df['Nombre']
    
    # Crear una nueva columna para indicar si la fila es correcta o tiene errores
    resultado = []
    
    for i, fila_rendicion in rendicion_df.iterrows():
        # Obtener la credencial y el nombre completo de la rendición
        credencial_rendicion = fila_rendicion['Cred']
        nombre_rendicion = fila_rendicion['Apellido y Nombres']
        
        # Buscar la credencial en el padrón
        fila_padron = padron_df[padron_df['Credencial'] == credencial_rendicion]
        
        # Verificar si la credencial existe y si el nombre coincide
        if not fila_padron.empty:
            nombre_padron = fila_padron.iloc[0]['Apellido y Nombre']
            
            # Debug: mostrar la comparación
            print(f"Comparando: {nombre_padron} (padrón) == {nombre_rendicion} (rendición)")
            
            if contar_coincidencias(nombre_rendicion, nombre_padron):
                resultado.append("Correcto")
            else:
                print(f"Error en nombre: {nombre_padron} != {nombre_rendicion}")  # Debug
                resultado.append("Error en nombre")
        else:
            print(f"Error en credencial: {credencial_rendicion} no se encontró en el padrón")  # Debug
            resultado.append("Error en credencial")
    
    # Agregar la columna de resultado al DataFrame de rendición
    rendicion_df['Resultado'] = resultado
    
    # Guardar el resultado en un nuevo archivo Excel
    rendicion_df.to_excel('RENDICON VENTAS COD608 P05-24 (CONTROLADO).xlsx', index=False)
    
    return rendicion_df

# Función principal que encapsula todo el proceso
def ejecutar_proceso(padron_file, rendicion_file):
    # Cargar los archivos
    padron_df, rendicion_df = cargar_archivos(padron_file, rendicion_file)
    
    # Procesar los DataFrames y hacer la comparación
    rendicion_controlada_df = procesar_comparacion(padron_df, rendicion_df)
    
    print("Proceso completado. Archivo guardado como 'RENDICON VENTAS COD608 P05-24 (CONTROLADO).xlsx'.")
    print(rendicion_controlada_df.head())  # Mostrar una vista previa del archivo resultante


In [ ]:
# Llamar a la función principal pasando los archivos
padron_file = 'PADRON ACTIVIDAD 21-05-24.xlsx'
rendicion_file = 'RENDICON VENTAS COD608 P05-24 (1).xlsx'

# Ejecutar el proceso
ejecutar_proceso(padron_file, rendicion_file)